# Import Important Libraries & Load dataset

In [ ]:
import pandas as pd
import numpy as np
from sklearn.utils import resample
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import json
from tensorflow.keras.preprocessing.text import tokenizer_from_json
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
import tensorflow as tf
from tensorflow.keras import Model, Input
from tensorflow.keras.layers import LSTM, Embedding, Dense, SimpleRNN, GRU
from tensorflow.keras.layers import TimeDistributed, SpatialDropout1D, Bidirectional
from keras.models import load_model

In [12]:
train = pd.read_csv('preprocessed_train_with_stopwords.csv', index_col=0)
train = train.drop(columns='Unnamed: 0', errors='ignore')
train

,label
text,
شركة الشياطين اليتريك من صبح ربي ال هارب لتوا,LY
يا عم ما قريبين من الدر برضو و عم حسن جوسي لو عتر علينا هيقتلنا,EG
يا جدعان لو بتحاول تبعتلي دي إم و إنت عامل بروتكتد أكاونت مش هاشوف حاجة و النعمة مش شايف غير إن عندي رسالة لا شايفها و لا أعرف صاحبها,EG
ما شفتش التزام تكتيكي للزمالك من سنين زيه رغم انه ملحقتش يقعد,EG
في أمان الله تصبحون على ما تحبون,MA
...,...
امالا كيفك تحبيط وبس,LY
العتب عالنضارة اللي مش لاقيها يا بنتي,EG
ده في كورة الجول ادهاله في رجله و هو صعب عليه راح مدهاله تاني انت بتقول ايه,EG


In [17]:
test = pd.read_csv(r"C:\Users\alabdeen\Desktop\Ai-Book\Advanced NLP\Project_nlp\preprocessed_test_with_stopwords.csv", index_col=0)
test.drop(columns='Unnamed: 0', inplace=False, errors='ignore')
test

,label
text,
وزى ما احنا وثقين فيك وبنحبك وشيفين اخلاصك وتعبك خليك واثق فينا احنا وراك باروحنا وثابتين وكلنا متقويين ببعض اطمن ياريس اثق بك ياسيسي,EG
هه أنا بس اللي مودي متغيرر,EG
ورقه معلقه لطلبة الطب فى احدى الجامعات الأجنبية خطرها على الغش اللي صاير عندكم,LY
صافي بالحلوف ايلا عودت درتها,MA
الدي ام بتاعك لو اتنشر يتعمل مسلسلات و تكسبي فلوس يالا مشروع عالجاهز اهوه,EG
...,...
ياراجل برا ارقد يوم كامل ولو صارت حاجة اني انوضك,LY
مالهم ناس الرباط التحضر وكل واحد حر في حياتو وراسو,MA
يمكن لأني كنت غبيه والله اعلم طلع ما يساهل مع الأسف,LY


In [18]:
# test = pd.read_csv('preprocessed_test_with_stopwords.csv')
# test.drop(columns='Unnamed: 0', inplace=True)
# test

# Tokenization


In [19]:
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer

# Convert NaN values to empty strings and ensure all entries are strings
train['text'] = train['text'].fillna('').astype(str)
test['text'] = test['text'].fillna('').astype(str)

# Initialize and fit the tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(train['text'])

train_sequences = tokenizer.texts_to_sequences(train['text'])
test_sequences = tokenizer.texts_to_sequences(test['text'])






KeyError: 'text'

# saving tokenizer


In [5]:
tokenizer_json = tokenizer.to_json()
with open('tokenizer.json', 'w', encoding='utf-8') as f:
    f.write(tokenizer_json)

In [6]:
# Load the tokenizer from the JSON file
with open('tokenizer.json', 'r', encoding='utf-8') as f:
    tokenizer_json = f.read()

# Convert JSON string back to tokenizer configuration
tokenizer = tokenizer_from_json(tokenizer_json)

# finding the max length for padding


In [7]:
max_sequence_len = 0
for sentence in train_sequences:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

67


In [8]:
max_sequence_len = 0
for sentence in test_sequences:
    max_sequence_len = max(len(sentence), max_sequence_len)
print(max_sequence_len)

59


In [9]:
max_sequence_length = 67
train_padded = pad_sequences(train_sequences, maxlen=max_sequence_length, padding='post')
test_padded = pad_sequences(test_sequences, maxlen=max_sequence_length, padding='post')



# Label Encoding


In [10]:
encoder = LabelEncoder()
y_encoded_train = encoder.fit_transform(train['label'])
y_encoded_train = to_categorical(y_encoded_train)
y_encoded_test = encoder.fit_transform(test['label'])
y_encoded_test = to_categorical(y_encoded_test)


In [11]:
y_encoded_train


array([[0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [1., 0., 0., 0., 0.],
       ...,
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.],
       [0., 1., 0., 0., 0.]])

# finding model input


In [12]:
num_words = len(tokenizer.word_index)
num_words


202143

# Models BY Pytorch

In [13]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

# Set device
device = torch.device('cpu')
# Convert labels to class indices instead of one-hot encoding
train_padded = train_padded.astype(int)
test_padded = test_padded.astype(int)
y_encoded_train = encoder.fit_transform(train['label'])
y_encoded_test = encoder.transform(test['label'])
# Convert data to PyTorch tensors
# Convert data to PyTorch tensors
X_train = torch.tensor(train_padded, dtype=torch.long)
X_test = torch.tensor(test_padded, dtype=torch.long)
y_train = torch.tensor(y_encoded_train, dtype=torch.long)
y_test = torch.tensor(y_encoded_test, dtype=torch.long)

# Move tensors to the device
X_train, X_test = X_train.to(device), X_test.to(device)
y_train, y_test = y_train.to(device), y_test.to(device)

# Create DataLoader for batch processing
batch_size = 64
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Hyperparameters
embedding_dim = 128
hidden_dim = 256
output_dim = len(encoder.classes_)  # Number of unique classes
vocab_size = num_words + 1
num_epochs = 10
learning_rate = 0.001

# Debugging shapes
print("X_train shape:", X_train.shape)  # Should be (num_samples, max_sequence_length)
print("y_train shape:", y_train.shape)  # Should be (num_samples,)
print("Max token index in train_padded:", train_padded.max())
print("Vocab size:", vocab_size)




X_train shape: torch.Size([118180, 67])
y_train shape: torch.Size([118180])
Max token index in train_padded: 202143
Vocab size: 202144


# Building RNN Model


In [14]:
class TextRNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TextRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.rnn = nn.RNN(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, _ = self.rnn(x)
        out = out[:, -1, :]  # Use the last hidden state
        out = self.fc(out)
        return out

In [15]:
class TextLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TextLSTM, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, (hidden, _) = self.lstm(x)
        out = hidden[-1]  # Use the last hidden state
        out = self.fc(out)
        return out

In [16]:
class TextGRU(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim):
        super(TextGRU, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        out, hidden = self.gru(x)
        out = hidden[-1]  # Use the last hidden state
        out = self.fc(out)
        return out

In [17]:
# Initialize models
rnn_model = TextRNN(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
lstm_model = TextLSTM(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
gru_model = TextGRU(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)

# Define loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizers = {
    "RNN": optim.Adam(rnn_model.parameters(), lr=learning_rate),
    "LSTM": optim.Adam(lstm_model.parameters(), lr=learning_rate),
    "GRU": optim.Adam(gru_model.parameters(), lr=learning_rate)
}

# Training function
def train_model(model, optimizer, criterion, train_loader, num_epochs):
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for X_batch, y_batch in train_loader:
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss:.4f}")

# Train RNN, LSTM, and GRU models
print("Training RNN...")
train_model(rnn_model, optimizers["RNN"], criterion, train_loader, num_epochs)

Training RNN...
Epoch 1/10, Loss: 2703.3040
Epoch 2/10, Loss: 2711.7498
Epoch 3/10, Loss: 2706.0382
Epoch 4/10, Loss: 2703.5895
Epoch 5/10, Loss: 2706.2555
Epoch 6/10, Loss: 2703.8636
Epoch 7/10, Loss: 2705.3050
Epoch 8/10, Loss: 2706.4720
Epoch 9/10, Loss: 2706.0240
Epoch 10/10, Loss: 2709.0732


In [18]:
torch.save(rnn_model.state_dict(), "rnn_model.pth")


In [25]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience  # Number of epochs to wait before stopping
        self.min_delta = min_delta  # Minimum change to qualify as an improvement
        self.best_loss = None  # Best validation loss seen so far
        self.best_epoch = 0  # Epoch corresponding to the best validation loss
        self.counter = 0  # Counter for epochs with no improvement

    def __call__(self, val_loss, model):
        # If this is the first epoch, initialize best_loss
        if self.best_loss is None:
            self.best_loss = val_loss

        # Check if the validation loss improved
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.best_epoch = epoch
            self.counter = 0  # Reset counter if there's an improvement
            # Save the model with the best validation loss
            torch.save(model.state_dict(), "best_model.pth")
        else:
            self.counter += 1
            if self.counter >= self.patience:
                print(f"Early stopping triggered at epoch {epoch + 1}")
                return True  # Stop training if no improvement for 'patience' epochs

        return False  # Continue training


In [21]:
def evaluate_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    total_loss = 0
    correct_predictions = 0
    total_samples = 0

    with torch.no_grad():  # No need to calculate gradients during evaluation
        for X_batch, y_batch in test_loader:
            # Move the data to the appropriate device
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            # Forward pass
            outputs = model(X_batch)
            
            # Calculate loss
            loss = criterion(outputs, y_batch)
            total_loss += loss.item()

            # Get the predicted class
            _, predicted = torch.max(outputs, 1)

            # Count correct predictions
            correct_predictions += (predicted == y_batch).sum().item()
            total_samples += y_batch.size(0)

    avg_loss = total_loss / len(test_loader)
    accuracy = (correct_predictions / total_samples) * 100

    print(f"Test Loss: {avg_loss:.4f}")
    print(f"Test Accuracy: {accuracy:.2f}%")
    
    return avg_loss, accuracy




In [28]:
# Assuming your LSTM model is already defined as lstm_model
# Define the optimizer
optimizer = torch.optim.Adam(lstm_model.parameters(), lr=0.001)

# Early stopping instance
early_stopping = EarlyStopping(patience=5, min_delta=0.001)

# Training loop with early stopping
num_epochs = 50  # Set the maximum number of epochs
for epoch in range(num_epochs):
    # Train model (make sure train_model function exists)
    train_loss = train_model(lstm_model, optimizer, criterion, train_loader,num_epochs)

    # Evaluate model
    val_loss, val_accuracy = evaluate_model(lstm_model, test_loader, criterion)

    # Check for early stopping
    if early_stopping(val_loss, lstm_model):
        break  # Exit the loop if early stopping is triggered

    # Print progress
    print(f"Epoch {epoch + 1}/{num_epochs}, Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%")

# Load the best model after training
best_model = TextLSTM(vocab_size, embedding_dim, hidden_dim, output_dim).to(device)
best_model.load_state_dict(torch.load("best_model.pth"))
best_model.eval()  # Set to evaluation mode


Epoch 1/50, Loss: 2686.9237
Epoch 2/50, Loss: 2684.3595


KeyboardInterrupt: 

In [ ]:
print("Training LSTM...")
train_model(lstm_model, optimizers["LSTM"], criterion, train_loader, num_epochs)

In [ ]:
torch.save(lstm_model.state_dict(), "lstm_model.pth")


In [ ]:
print("Training GRU...")
train_model(gru_model, optimizers["GRU"], criterion, train_loader, num_epochs)

In [ ]:
torch.save(gru_model.state_dict(), "gru_model.pth")


In [24]:
print("Evaluating RNN...")
evaluate_model(rnn_model, test_loader, criterion)


Evaluating RNN...
Test Loss: 1.4597
Test Accuracy: 39.02%


(1.4596708850427107, 39.01506177018108)

In [ ]:

print("Evaluating LSTM...")
evaluate_model(lstm_model, test_loader)

In [ ]:

print("Evaluating GRU...")
evaluate_model(gru_model, test_loader)